Build an inventory management system for a supermarket
- Create add product to inventory function
- Create remove product from inventory function 
- Create search product function
- Create low quantity in inventory function
- Create view Inventory function

In [ ]:
# import needed libraries
import pandas as pd
import re
from datetime import datetime
import time

In [ ]:
#create a data log function for all actions performed on this program
def data_log_func (action):
    """
    creates a log file to register every addition or subraction that happened on the inventory file
    """
    try:
        with open ('data_log.csv','a') as f:
            f.write(f'{action},{ datetime.now().time()},{ datetime.now().date()}\n')
    except Exception:
        print('Passed')

In [ ]:
def quan_or_price(x = 'input value: '):
    while True:
        """
        input prompt to accept numerical input only for quantity and price
        """
        try:
            quantity = input(x)
            if quantity.strip() == '00': 
                break
            else:
                return int(quantity)
        except ValueError :
            print('Input numeric values\n')

In [ ]:
def input_prompt():
    # make last quantity input and last price input accessible anywhere
    global quantity
    global price
    '''
            1. function to reduce repetition of writing codes
            2. accepts input for quantity and price
            3. inputs can be accessed anywhere
    '''
    try:
        quantity = quan_or_price('Enter quantity: ')
        price = quan_or_price('Enter price: ')       
        return int(quantity), float(price)
    except TypeError:
        return quantity, price

In [ ]:
# adding products to inventory  
def add_product():
    """
    inputs product name, quantity, price, datetime of inventory to inventory data file
    """
        # input product name
    product_name = input('Enter Product name: ').lower()
    # check if inventory data file already exist
    try:
        data = pd.read_csv('inventory_data.csv',names = ['Product Name', 'Quantity','Price','Datetime'])
        # search product in dataset
        existing_product = data[data['Product Name'].str.lower() == product_name]
        
        
        if not existing_product.empty: # to update if product name already exist
            print(product_name, 'already exist')
            update_product = input('Do you want to update? (Y/N): ').upper()
            # to update already existing product name
            if update_product == 'Y':
                # user inputs new quantity and new price
                input_prompt()
                # input new quantity, new price and datetime
                if product_name is not None and quantity is not None and price is not None:
                    print(quantity is not None,price)
                    data.loc[data['Product Name'] == product_name,['Quantity','Price','Datetime']] = (quantity, price, datetime.now())
                  
                    data.to_csv('inventory_data.csv',header  = None, index = False) # return dataset as csv file
                    data_log_func(f"updated '{product_name}'")
                    
                    print(f'{product_name}', 'updated')
                else:
                    print('Operation Cancelled')
            else: # if user does not want to update product
                print('No Product Updated')
            
        else: # if there is product name search mismatch in the product name
            searched = search_product_name(product_name)
            if searched:
                update_product = input('Do you want to update? (Y/N)').upper()
                if update_product == 'Y':
                              # input new quantity, new price and datetime
                    input_prompt()
                    if product_name is not None and quantity is not None and price is not None:
                        data.loc[data['Product Name'] == searched, ['Quantity','Price','Datetime']] = (quantity, price, datetime.now())
                                  # return dataset as csv file
                        data.to_csv('inventory_data.csv',header  = None, index = False)
                        # data_log_func(f"updated '{productName}'")
                        data_log_func(f"updated '{searched}'")
                        print(f'{searched}', 'updated in Inventory')
                    else:
                        print('Operation cancelled')
                else:
                    print('No Product Updated')
            else:
                input_prompt()
                if product_name is not None and quantity is not None and price is not None:
                    # print(product_name is None and quantity is None and price is None)
                    '''join the old dataset and the new product name together'''
                    new_data = pd.DataFrame({'Product Name':[product_name],'Quantity':[quantity],\
                                    'Price':[price], 'Datetime':[datetime.now()]})
                    data = pd.concat([data, new_data], axis = 0, ignore_index = True)
                    data.to_csv('inventory_data.csv', header = None, index = False)
                    data_log_func(f"updated '{product_name}'")
                    print(f'{product_name}', 'updated to Inventory')
                    
                else:
                    print('Operation Cancelled!')
    except FileNotFoundError: #if file runs through File Not Found Error
        prompt_user = input('Do you want to create a new Inventory file: (Y/N)').upper()
        if prompt_user == 'Y':
            print('Inventory file created')
            input_prompt()
            if product_name is not None and quantity is not None and price is not None:
                data = pd.DataFrame({'Product Name':[product_name],'Quantity':[quantity],'Price':[price], 'Datetime':[datetime.now()]})
                data.to_csv('inventory_data.csv', index = False, header = None)
                data_log_func(f"added '{product_name}'")
            else:
                print('Operation Cancelled')

        else:
            print('File does not exist')
         # to show add product process complete
    print('Done!')

In [ ]:
def check_num(prompt = 'input option'):
    """
    check if the input value is a numeric value    
    """
    while True: # continue running if the input value is not a numeric value
        try:
            input_option = input(prompt).strip()
            int_input_option = int(input_option) 
            return input_option, int_input_option - 1
        except ValueError:
            print('input numerical value')

In [ ]:
#search word suggestion function
def search_product_name(x, exit_message = 'Cancelled'):
    if len(x) > 0 and x is not None:
        data = pd.read_csv('inventory_data.csv',\
                            names = ['Product Name', 'Quantity','Price','Datetime'])
        search_word = data['Product Name'] # get series data of product names available
        pattern = ''.join(['(?=.*' + letter + '.*)' for letter in x]) # create different search patterns for search_product_name function
        pattern += '.{2,}' # patterns uses 2 or more word for search_product_name function
        matches = [word for word in search_word if re.search(pattern, word)] # result of all completed suggestions matches
        
        if len(matches) > 0: # if length of matches list is greater than 0
            print('These suggestions came out. \nDo you mean:')
            for index, word in enumerate(matches, start=1):
                print(f'{index}. {word}') # shows suggestions
            print('00. Exit / Cancel') # to exit
        
        # run a while loop to select option out of matches list
        while len(matches)>0 : # if length of matches list is greater than 0
            input_option, int_input_option = check_num('Please input a numerical option: ')           

            if 0 <= int_input_option < len(matches) :
                print(f"You selected: {matches[int_input_option]}")
                return matches[int_input_option]
                break
            elif re.match(r"^00$", input_option):
                print(exit_message)
                break
            else:
                print('Invalid selection. Please choose a correct option.')

In [ ]:
def view_inventory():
    """
    view all inventory function
    """
    try:
      # read csv file  
        data = pd.read_csv('inventory_data.csv', header=None, names=['Product Name', 'Quantity', 'Price', 'Datetime'])
        # if data is empty
        if data.empty:
            print('No products in the inventory')
        else: # display all inventory table
            print('\n')
            print('Product Inventory: ')
            return display(data)
    except FileNotFoundError: # run if file runs through  a file not found error
        print('File does not exist\n')

In [ ]:
def search_product():
    """
    function searches for a product in inventory data file
    """    
    product_name = input('Enter the name of the product to search: ').lower()
    try:
      # reads inventory file csv
        data = pd.read_csv('inventory_data.csv', names=['Product Name', 'Quantity',\
                                     'Price','Datetime'])
        product_name_check = data[data['Product Name'] == product_name]
        if not product_name_check.empty: 
          # returns product name if product name matches a product name in the dataset
            print(f'"{product_name}" found\n')
            display(product_name_check)

        else:
          # search product_name with suggestions
            searched = search_product_name(product_name,exit_message = '')
            if searched: # runs if there is one or more suggestions
                display (data[data['Product Name'] == searched])
            else:
                print(f'{product_name}', 'not found in Inventory')
    except FileNotFoundError: # run if file runs through  a file not found error
        print('\nFile does not exist\n')

In [ ]:
# remove product name from inventory file list function
def remove_product_name(x = ''):
    try: 
        with open('inventory_data.csv','r') as f:
            file = f.read() 
        file_split = file.splitlines()
        file_split = [word.split(',') for word in file_split]
        file_split_ = ''
        if x == 'wipeWI__PEwipe':
            file_split_= ''
            data_log_func('inventory wiped')
            print('File inventory Wiped')
        else:
            product_name = input('Enter product name to be removed: ').lower().strip() # remove border spaces and trun to lower case strings
            # open inventory data csv file to read
            for index,item in enumerate(file_split):
                if item[0] == product_name:
                    file_split.pop(index)            
                    file_split_ = '\n'.join(_ for _ in [', '.join (word) for word in file_split])# joins elements in every list together and convert all to strings
                    data_log_func(f"'removed', {product_name}")
                    print( f'"{product_name}"','removed from inventory')
                    break
            else:
                # use search_product_name to get suggestions
                searched = search_product_name(product_name)
                if searched:
                    for index,item in enumerate(file_split):
                        if item[0] == searched:
                            file_split.pop(index)
                            file_split_ = '\n'.join(_ for _ in [', '.join (word) for word in file_split]) # joins elements in every list together and convert all to string
                            print(f'"{searched}"', 'removed from inventory') # outputs after the searched is done
                            break
                else:
                    file_split_ = '\n'.join(_ for _ in [', '.join (word) for word in file_split]) # joins elements in every list together and convert all to strings 
                    print(product_name,'is not found')
        try:
            with open('inventory_data.csv','w') as f_: # write csv file again after removal of product name
                f_.write(file_split_)
                print('\nDone!!!')
        except TypeError: # return if file is empty
            print('\nFile is empty')
    except FileNotFoundError :
        print('\nFile does not exist!')

In [ ]:
def greet():
    """
    Salutation for inventory menu
    """
    now = datetime.now()
    if now.hour < 12: # time from 12am to 11:59 am
        return 'Good morning!'
    elif now.hour < 17 : # time from 12pm to 4:59 pm
        return 'Good afternoon!'
    else: # time from 5pm to 11:59 pm
        return 'Good evening!'

In [ ]:
def low_notification(low = 5):
    """
    checks if the quantity is lower than the threshold provided
    """
    print('low quantity items: ')
    try:
        data = pd.read_csv('inventory_data.csv',names = ['Product Name', 'Quantity','Price','Datetime'])
        low_items = data.loc[data['Quantity'] < 5 ,['Product Name','Quantity']].reset_index(drop = True)
        display(low_items)
    except FileNotFoundError :
        print('File does not exist')

In [ ]:
def run_inventory_ms():
    print(greet())
    count = 0
    while True:
        # interactive input options to be selected
        print('\nInventory Management System')
        print('1. Add Product')
        print('2. View Inventory')
        print('3. Search Product')
        print('4. Remove Inventory')
        print('00. Exit')
        time.sleep(0.1)
        choice = input('Enter your choice: ')
        if choice == '1':
             add_product()
        elif choice == '2':
             view_inventory()
        elif choice == '3':
             search_product()
        elif choice == '4':
            remove_product_name()
        elif choice == '00':
            print('Exit!')
            break
        else:
            print('\nInvalid choice. please enter a number from the list above.')
        count += 1
        if count % 4 == 0:
            low_notification()

In [ ]:
if __name__ == '__main__':
    run_inventory_ms()